# Dilithium - PolyGlitches

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'
NIST_SECURITY_LEVEL = 2
NUM_ITERS = 64

In [2]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger('polyglitches')

In [3]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
if 'python-dilithium' not in sys.path:
    sys.path.append('python-dilithium')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software',
 'python-dilithium']

In [4]:
import chipwhisperer as cw
import importlib
import json
import uuid
import numpy as np
from collections import defaultdict
from dilithium import Dilithium
from chipwhisperer.capture.targets import TargetIOError, TargetTimeoutError
d = Dilithium(NIST_SECURITY_LEVEL)
cw, d

(<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>,
 <dilithium.Dilithium at 0x7fc44818ba00>)

In [5]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [6]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [7]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

In [8]:
#cw.program_target(scope, prog, fw_path)

In [9]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]

In [10]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.clock.adc_src = "clkgen_x1" # we do not need x4 as we do not capture traces; maybe speedup?
    scope.adc.samples = 0 # maybe speedup?
    # scope.adc.timeout = .002
    scope.adc.timeout = .1

def get_duration():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.loop(20)
    timed_out = scope.capture()
    assert not timed_out
    return scope.adc.trig_count;

def get_poly_correct_packed() -> bytes:
    dilithium2_polyz_unpack_settings()
    target.loop()
    poly = target.get_poly()
    return poly

def get_poly_correct() -> np.ndarray:
    poly_packed = get_poly_correct_packed()
    return d._polyz_unpack(poly_packed)

dilithium2_polyz_unpack_settings()
duration = get_duration()
poly_correct = get_poly_correct()
print(f'clock cycle duration all = {duration}')
print(f'clock cycle duration iteration = {duration / NUM_ITERS}')
poly_correct

clock cycle duration all = 4698
clock cycle duration iteration = 73.40625


array([-100330,   65070,    9596,   57850,   35652,  105533,  -12476,
        -46863,  -37129,  -56173,  -21068,    2324,  115540,   44633,
        125101,    3586,   69993,  -81687,  -64128,  120572,    2307,
        130526,    4041,  -41650,   84699,   -8418,   74822,  109619,
        -91410,  -91558,  115894,   23794,   38581,   97367,  -68344,
         13693,  -29260,  -93738,   25493,   28466,  -98993,  -35421,
       -129186,  -35972,  -98782,    5229,  121668,  -36138,   43042,
        126578,  -51112,  -26829,   88126,  -48827,   -6483,   91640,
         26814,  122947,   26573,   69248,   24721,  -59507,   95958,
         40139,  -14569,   50396, -112991,  -68730,   66400,  -78235,
        -86549,  -76860,   64682,  -31281,  -67048, -120548,  -94141,
        -16141,  -41256, -107494,  123510,   92927,  -46082, -106954,
        123712,  -54074,  -68055,  -27761,   95540,  -34279,   88614,
        -16752, -106508,  110918,  -79361, -111904,  -61989,  -37948,
        -98516,  -55

In [11]:
def get_iter(faulted_poly: np.ndarray, correct_poly: np.ndarray = poly_correct, coeffs_per_iter: int = 4) -> bool:
    """
    0 = all coeffs zero; 1 - 63 = some non faulted in the beginning rest zero;
    64 = no fault; None = not our fault model (i.e. faults within iters)
    """
    is_same = faulted_poly == correct_poly
    is_zero = faulted_poly == 0
    for i in range(0, NUM_ITERS + 1):
        # index of the first zero coefficient due to a fault
        split_index = i * coeffs_per_iter
        
        same_as_expected = np.all(is_same[:split_index])
        if not same_as_expected:
            return None
        rest_faulted = np.all(is_zero[split_index:])
        
        if same_as_expected and rest_faulted:
            return i
    return None

In [12]:
glitch_spots = [299]
width_start = 6.640625
width_stop = 6.640625
offset_start = 6.640625
offset_stop = 6.640625
repeat = 1
step = 1
redo = 10
is_faulted = lambda candidate_poly: not np.all(candidate_poly == correct_poly)
(299, 6.640625, 6.640625)

(299, 6.640625, 6.640625)

In [13]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=[f"perfect_{i}" for i in range(NUM_ITERS)] + ["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()

IntText(value=0, description='perfect_0 count:', disabled=True)

IntText(value=0, description='perfect_1 count:', disabled=True)

IntText(value=0, description='perfect_2 count:', disabled=True)

IntText(value=0, description='perfect_3 count:', disabled=True)

IntText(value=0, description='perfect_4 count:', disabled=True)

...

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

In [14]:
faulted_polys = []
faulted_poly_params = []
perfect_db = defaultdict(lambda: [])

In [15]:
scope.glitch.repeat = repeat
gc.set_range("width", width_start, width_stop)
gc.set_range("offset", offset_start, offset_stop)
gc.set_global_step(step)
target.reboot_flush()

b'\x00\x0bb\x07boot ok\xc1\x00'

In [16]:
def one_try(scope, target, gc, is_faulted, faulted_polys, faulted_polys_params) -> bool:
    scope.arm()
    try:
        target.loop_send()
        
        trigger_timeout = scope.capture()
        if trigger_timeout:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            __LOGGER.info('fail due to trigger timeout')
            target.reboot_flush()
            return
        
        target.loop_recv(timeout=10)
    except TargetTimeoutError as e:
        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
        __LOGGER.info(f'failed due to read timeout: {e}')
        target.reboot_flush()
        return
    except TargetIOError as e:
        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
        __LOGGER.info(f'failed due to TargetIOError error: {e}')
        target.reboot_flush()
        return
    else:
        try:
            poly_candidate_packed = target.get_poly(max_num_retries=10)
            poly_candidate = d._polyz_unpack(poly_candidate_packed)
            print(poly_candidate)
            
            i = get_iter(poly_candidate)
            if i is None:
                gc.add("success", (scope.glitch.width, scope.glitch.offset))
            elif i == NUM_ITERS:
                gc.add("normal", (scope.glitch.width, scope.glitch.offset))
                return
            elif 0 <= i < NUM_ITERS:
                gc.add(f"perfect_{i}", (scope.glitch.width, scope.glitch.offset))
            else:
                __LOGGER.warning('Unexpected iteration!')
            
            __LOGGER.info("Glitched! Ext offset: {} Offset: {} tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            
            faulted_polys.append(poly_candidate)
            faulted_poly_params.append((scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
        except TargetIOError as e:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            __LOGGER.info(f'Failed to read poly: {e}')
            target.reboot_flush()

In [17]:
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in glitch_spots:
        scope.glitch.ext_offset = i
        for j in range(redo):
            one_try(scope, target, gc, is_faulted, faulted_polys, faulted_poly_params)

INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625
INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625


[-100330   65070    9596   57850   35652  105533  -12476  -46863  -37129
  -56173  -21068    2324  115540   44633  125101    3586   69993  -65536
  -64128  120572    2307  130526    4041  -41650   84699   -8418   74822
  109619  -91410  -91558  115894   23794   38581   97367  -68344   13693
  -29260  -93738   25493   28466  -98993  -35421 -129186  -35972  -98782
    5229  121668  -36138   43042  126578  -51112  -26829   88126  -48827
   -6483   91640   26814  122947   26573   69248   24721  -59507   95958
   40139  -14569   50396 -112991  -68730   66400  -78235  -86549  -76860
   64682  -31281  -67048 -120548  -94141  -16141  -41256 -107494  123510
   92927  -46082 -106954  123712  -54074  -68055  -27761   95540  -34279
   88614  -16752 -106508  110918  -79361 -111904  -61989  -37948  -98516
  -55493   39202  -40934  -32598   76273   51232  -41914   65442  -16213
   55769   94113   50226  120092 -128689  -94408  -61484  -19585  -88952
  -76813    2824   91206  -56337 -118806  -96835   

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:polyglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625
INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625
INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625


[-100330   65070    9596   57850   35652  105533  -12476  -46863  -37129
  -56173  -21068    2324  115540   44633  125101    3586  130948  -65559
  -64128  120572    2307  130526    4041  -41650   84699   -8418   74822
  109619  -91410  -91558  115894   23794   38581   97367  -68344   13693
  -29260  -93738   25493   28466  -98993  -35421 -129186  -35972  -98782
    5229  121668  -36138   43042  126578  -51112  -26829   88126  -48827
   -6483   91640   26814  122947   26573   69248   24721  -59507   95958
   40139  -14569   50396 -112991  -68730   66400  -78235  -86549  -76860
   64682  -31281  -67048 -120548  -94141  -16141  -41256 -107494  123510
   92927  -46082 -106954  123712  -54074  -68055  -27761   95540  -34279
   88614  -16752 -106508  110918  -79361 -111904  -61989  -37948  -98516
  -55493   39202  -40934  -32598   76273   51232  -41914   65442  -16213
   55769   94113   50226  120092 -128689  -94408  -61484  -19585  -88952
  -76813    2824   91206  -56337 -118806  -96835   

INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625
INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625
INFO:polyglitches:Glitched! Ext offset: 299 Offset: 6.640625 tWidth: 6.640625


[-100330   65070    9596   57850   35652  105533  -12476  -46863  -37129
  -56173  -21068    2324  115540   44633  125101    3586  130948  -65559
  -64128  120572    2307  130526    4041  -41650   84699   -8418   74822
  109619  -91410  -91558  115894   23794   38581   97367  -68344   13693
  -29260  -93738   25493   28466  -98993  -35421 -129186  -35972  -98782
    5229  121668  -36138   43042  126578  -51112  -26829   88126  -48827
   -6483   91640   26814  122947   26573   69248   24721  -59507   95958
   40139  -14569   50396 -112991  -68730   66400  -78235  -86549  -76860
   64682  -31281  -67048 -120548  -94141  -16141  -41256 -107494  123510
   92927  -46082 -106954  123712  -54074  -68055  -27761   95540  -34279
   88614  -16752 -106508  110918  -79361 -111904  -61989  -37948  -98516
  -55493   39202  -40934  -32598   76273   51232  -41914   65442  -16213
   55769   94113   50226  120092 -128689  -94408  -61484  -19585  -88952
  -76813    2824   91206  -56337 -118806  -96835   

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:polyglitches:failed due to read timeout: Target cleanly timed out while generating a signature


In [18]:
print(i)

299


In [19]:
result_data = {
    "i": i,
    "poly_correct": poly_correct.tolist(),
    "glitch_spots": glitch_spots,
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "step": step,
    "faulted_polys": list(map(lambda np_poly: np_poly.tolist(), faulted_polys)),
    "faulted_polys_params_descropton": "0 = ext_offset, 1 = offset, 2 = width",
    "faulted_polys_params": faulted_poly_params
}
with open(f'77param{uuid.uuid4()}.json', 'w') as f:
    json.dump(result_data, f)

In [20]:
scope.dis()
target.dis()